# Comparison between GraphRNN and GRAN

## Setup

In [13]:
import os
import sys
import torch
import logging
import traceback
import numpy as np
from pprint import pprint
import pandas as pd
from runner.train_runners import *
from utils.logger import setup_logging
from utils.arg_helper import parse_arguments, get_config
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
torch.set_printoptions(profile='full')


## Useful Functions

In [14]:
df = pd.read_csv("save_model_learning.csv")
df

,Unnamed: 0,Date,dataset_name,dataset_num,model_name,num_epochs,file_dir
0,0,2021-Aug-15-01-56-10,community2,500,GRANMixtureBernoulli,5,exp/GRAN/community2\GRANMixtureBernoulli_commu...
1,0,2021-Aug-15-02-09-00,community4,500,GRANMixtureBernoulli,5,exp/GRAN/community4\GRANMixtureBernoulli_commu...
2,0,2021-Aug-15-02-29-46,community8,500,GRANMixtureBernoulli,5,exp/GRAN/community8\GRANMixtureBernoulli_commu...
3,0,2021-Aug-16-14-28-27,watts,500,GRANMixtureBernoulli,5,exp/GRAN/watts\GRANMixtureBernoulli_watts_2021...
4,0,2021-Aug-17-16-56-29,barabasi,500,GRANMixtureBernoulli,15,exp/GRAN/barabasi\GRANMixtureBernoulli_barabas...
5,0,2021-Aug-17-22-00-39,barabasi,500,GRANMixtureBernoulli,50,exp/GRAN/barabasi\GRANMixtureBernoulli_barabas...
6,0,2021-Aug-18-13-17-52,community2,500,RNN,1000,exp/GraphRNN/rnn/community2\RNN_community2_202...
7,0,2021-Aug-18-16-05-01,community4,500,RNN,1000,exp/GraphRNN/rnn/community4\RNN_community4_202...
8,0,2021-Aug-18-17-43-34,community4,500,RNN,1000,exp/GraphRNN/rnn/community4\RNN_community4_202...
9,0,2021-Aug-18-22-08-24,community8,500,RNN,1000,exp/GraphRNN/rnn/community8\RNN_community8_202...


## Research Questions
1) Which one of the model is the better for each dataset?

2) About GRAN, which node ordering gives better results ?

3) Is there a bias from GRAN to generates communities ? ( How many ? Scalability ? Robustness ?)

4) What are optimal M-parametes for GraphRNN for each dataset ?
Does the parameter tweaking change the efficiency significantly ?

5) Does the SotA autoregressive model are able to retain the small-world propertie from the graph ( = avg length btw 2 nodes is proportional to the log of the number of nodes N )
(create GNN classifier "small-world" or GAN)

## Experiments
####  1) Which one of the model is the better for each dataset?
####  2) About GRAN, which node ordering gives better results ?
####  3) Is there a bias from GRAN to generates communities ? ( How many ? Scalability ? Robustness ?)
####  4) What are optimal M-parametes for GraphRNN for each dataset ? Does the parameter tweaking change the efficiency significantly

## Datasets

### Erdos Renyi dataset
Parameters used : 500 graphs btw 100-200 nodes with p=0.1

Node ordering (GRAN) :

### Barabasi Albert Dataset
Parameters used :Parameters used : 500 graphs btw 100-200 nodes with k=4/5

Node ordering (GRAN) :

### Watts Strogatz Dataset
Parameters used :Parameters used : 500 graphs btw 100-200 nodes with p=0.01

Node ordering (GRAN) : DFS


### Community Dataset
Parameters used : graphs of 2/4/8 communities of between 12 to 17 nodes

Node ordering (GRAN) : DFS


## Results

In [15]:
def get_stats_from_trained_model(config):
    """Return all mmd statistical results from
    generated graph by the trained model, in the form of a dict"""

    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    torch.cuda.manual_seed_all(config.seed)
    config.use_gpu = config.use_gpu and torch.cuda.is_available()
    torch.cuda.empty_cache()

    runner = eval(config.runner)(config)

    mmd_degree_dev, mmd_clustering_dev, mmd_4orbits_dev, mmd_spectral_dev, mmd_degree_test, mmd_clustering_test, mmd_4orbits_test, mmd_spectral_test = runner.test()

    return {"mmd_degree_dev": mmd_degree_dev, "mmd_clustering_dev": mmd_clustering_dev,
            "mmd_4orbits_dev": mmd_4orbits_dev, "mmd_spectral_dev": mmd_spectral_dev,
            "mmd_degree_test": mmd_degree_test, "mmd_clustering_test": mmd_clustering_test,
            "mmd_4orbits_test": mmd_4orbits_test, "mmd_spectral_test": mmd_spectral_test}


row_list = []
for training_path in df['file_dir']:

    try :
        config_path = os.path.join(training_path, 'config.yaml')
        config = get_config(config_path)
    except :
        continue
    if training_path.find('mlp') == -1:
        dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name,
                    "num_epochs": config.train.max_epoch}
    else :
        dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name+"_MLP",
                    "num_epochs": config.train.max_epoch}

    dict_stats = get_stats_from_trained_model(config)
    dict_results.update(dict_stats)
    row_list.append(dict_results)
    torch.cuda.empty_cache()

result_df=pd.DataFrame(row_list)
torch.cuda.empty_cache()

max # nodes = 32 || mean # nodes = 32.0
max # edges = 170 || mean # edges = 162.574


 10%|█         | 2/20 [00:08<01:17,  4.29s/it]


KeyboardInterrupt: 

In [24]:
result_df


,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community2,GRANMixtureBernoulli,5,0.015923,0.202623,1.396051e-01,0.071799,0.017348,0.276299,1.712755e-01,0.073148
1,community4,GRANMixtureBernoulli,5,0.009925,0.036726,4.744825e-01,0.020330,0.008988,0.033917,4.378288e-01,0.019204
2,community8,GRANMixtureBernoulli,5,0.019024,0.021497,5.293842e-01,0.004585,0.019817,0.020867,4.909517e-01,0.005272
3,watts,GRANMixtureBernoulli,5,0.122652,0.030463,1.770889e-03,0.176546,0.122599,0.029367,1.759161e-03,0.206862
4,barabasi,GRANMixtureBernoulli,15,0.052985,0.075533,1.045794e-01,0.010339,0.051783,0.195645,8.814985e-02,0.009107
5,barabasi,GRANMixtureBernoulli,50,0.030012,0.041243,8.412683e-02,0.013379,0.031000,0.199665,7.644321e-02,0.012469
6,community2,RNN,1000,0.042362,0.234516,3.248894e-01,0.107387,0.043783,0.308299,3.263660e-01,0.110081
7,community4,RNN,1000,0.018449,0.067304,6.084830e-01,0.028228,0.018187,0.064511,5.924443e-01,0.028171
8,community8,RNN,1000,0.062338,0.057058,6.888835e-01,0.059540,0.059766,0.055865,6.393324e-01,0.059826
9,watts,RNN,1000,0.000029,0.000270,9.113732e-08,0.011266,0.000003,0.000158,7.558648e-09,0.028463


In [26]:
result_df.style

,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community2,GRANMixtureBernoulli,5,0.015923,0.202623,0.139605,0.071799,0.017348,0.276299,0.171276,0.073148
1,community4,GRANMixtureBernoulli,5,0.009925,0.036726,0.474483,0.020330,0.008988,0.033917,0.437829,0.019204
2,community8,GRANMixtureBernoulli,5,0.019024,0.021497,0.529384,0.004585,0.019817,0.020867,0.490952,0.005272
3,watts,GRANMixtureBernoulli,5,0.122652,0.030463,0.001771,0.176546,0.122599,0.029367,0.001759,0.206862
4,barabasi,GRANMixtureBernoulli,15,0.052985,0.075533,0.104579,0.010339,0.051783,0.195645,0.088150,0.009107
5,barabasi,GRANMixtureBernoulli,50,0.030012,0.041243,0.084127,0.013379,0.031000,0.199665,0.076443,0.012469
6,community2,RNN,1000,0.042362,0.234516,0.324889,0.107387,0.043783,0.308299,0.326366,0.110081
7,community4,RNN,1000,0.018449,0.067304,0.608483,0.028228,0.018187,0.064511,0.592444,0.028171
8,community8,RNN,1000,0.062338,0.057058,0.688883,0.059540,0.059766,0.055865,0.639332,0.059826
9,watts,RNN,1000,0.000029,0.000270,0.000000,0.011266,0.000003,0.000158,0.000000,0.028463


Here are the mmd (with emd) metrics results for each data
#### 2-Community Dataset stats

In [27]:
result_df[result_df['dataset_name']=="community2"].style.highlight_min(color = 'lightblue', axis = 0)

,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community2,GRANMixtureBernoulli,5,0.015923,0.202623,0.139605,0.071799,0.017348,0.276299,0.171276,0.073148
6,community2,RNN,1000,0.042362,0.234516,0.324889,0.107387,0.043783,0.308299,0.326366,0.110081
14,community2,RNN_MLP,1000,0.042169,0.234598,0.527127,0.125244,0.043783,0.308377,0.499491,0.130945


#### 4-Community Dataset stats

In [28]:
result_df[result_df['dataset_name']=="community4"].style.highlight_min(color = 'lightblue', axis = 0)

,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
1,community4,GRANMixtureBernoulli,5,0.009925,0.036726,0.474483,0.020330,0.008988,0.033917,0.437829,0.019204
7,community4,RNN,1000,0.018449,0.067304,0.608483,0.028228,0.018187,0.064511,0.592444,0.028171
15,community4,RNN_MLP,1000,0.050438,0.067263,0.576653,0.041477,0.050584,0.064461,0.560370,0.039798


#### 8-Community Dataset stats

In [29]:
result_df[result_df['dataset_name']=="community8"].style.highlight_min(color = 'lightblue', axis = 0)

,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
2,community8,GRANMixtureBernoulli,5,0.019024,0.021497,0.529384,0.004585,0.019817,0.020867,0.490952,0.005272
8,community8,RNN,1000,0.062338,0.057058,0.688883,0.059540,0.059766,0.055865,0.639332,0.059826
16,community8,RNN_MLP,1000,0.104196,0.059649,0.704969,0.067516,0.100600,0.058457,0.655418,0.067214


#### Barabasi Dataset stats

In [30]:
result_df[result_df['dataset_name']=="barabasi"].style.highlight_min(color = 'lightblue', axis = 0)

,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
4,barabasi,GRANMixtureBernoulli,15,0.052985,0.075533,0.104579,0.010339,0.051783,0.195645,0.088150,0.009107
5,barabasi,GRANMixtureBernoulli,50,0.030012,0.041243,0.084127,0.013379,0.031000,0.199665,0.076443,0.012469
10,barabasi,RNN,1000,0.150253,0.389317,0.268000,0.050266,0.149267,0.506531,0.254160,0.019408
11,barabasi,RNN_MLP,1000,0.159249,0.303681,0.286197,0.047200,0.158652,0.433172,0.250644,0.019461
12,barabasi,RNN_MLP,1000,0.137827,0.318624,0.262523,0.053429,0.138651,0.456399,0.224218,0.021015


#### Watts-Strogatz (Ring Graph mode) Dataset stats

In [25]:
result_df[result_df['dataset_name']=="watts"].style.highlight_min(color = 'lightblue', axis = 0)


,dataset_name,model_name,num_epochs,mmd_degree_dev,mmd_clustering_dev,mmd_4orbits_dev,mmd_spectral_dev,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
3,watts,GRANMixtureBernoulli,5,0.122652,0.030463,1.770889e-03,0.176546,0.122599,0.029367,1.759161e-03,0.206862
9,watts,RNN,1000,0.000029,0.000270,9.113732e-08,0.011266,0.000003,0.000158,7.558648e-09,0.028463
13,watts,RNN_MLP,1000,0.000090,0.000001,9.058890e-07,0.012680,0.000061,0.000003,7.519799e-07,0.023366
